In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
os.chdir(os.getcwd().rsplit('/',1)[0])

In [3]:
# nominate columns
col_tar = ['beer_style']
col_cat = ['brewery_name']
col_num = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste']

In [17]:
filepath = 'data/processed/beer_review_cleaned.csv'

# load data from csv
df = pd.read_csv('data/raw/beer_reviews.csv')

# clean up dataset: drop unrelated columns and drop rows that contain NA 
df_cleaned = df.copy()
df_cleaned = df[col_cat + col_num + col_tar]
# df_cleaned = df[col_num + col_tar]

# store data
df_cleaned.dropna(inplace=True)
df_cleaned.to_csv('data/processed/beer_review_cleaned.csv', index=False)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [20]:
from sklearn.model_selection import train_test_split

# remove brewery name as it's irrelevant
bco = df_cleaned.pop('brewery_name')

df_cleanedx = df_cleaned.copy()
df_cleanedx = df_cleanedx[df_cleanedx['beer_style']=='American IPA']

# extract target label column
target = df_cleanedx.pop(col_tar[0])

KeyError: 'brewery_name'

In [21]:
df_cleaned = df_cleanedx

In [19]:
df_cleanedx[col_tar[0]].value_counts()

American IPA    117584
Name: beer_style, dtype: int64

In [6]:
target.value_counts()

American IPA                        117584
American Double / Imperial IPA       85977
American Pale Ale (APA)              63469
Russian Imperial Stout               54129
American Double / Imperial Stout     50705
                                     ...  
Gose                                   686
Faro                                   609
Roggenbier                             466
Kvass                                  297
Happoshu                               241
Name: beer_style, Length: 104, dtype: int64

In [7]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
enc = LabelEncoder()
enc_target = np.array(target)
enc_target = enc_target.reshape(-1,1)
enc_target = enc.fit_transform(target)

In [8]:
enc.transform(['American IPA'])

array([12])

In [9]:
enc.inverse_transform([12])

array(['American IPA'], dtype=object)

In [10]:
import joblib

joblib.dump(enc, 'models/output_encoder.joblib')

['models/output_encoder.joblib']

In [11]:
enc_target.shape

(1586599,)

In [12]:
df_cleaned.describe()

,review_aroma,review_appearance,review_palate,review_taste
count,1.586599e+06,1.586599e+06,1.586599e+06,1.586599e+06
mean,3.735638e+00,3.841647e+00,3.743705e+00,3.792864e+00
std,6.976142e-01,6.160899e-01,6.822131e-01,7.319658e-01
min,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00
50%,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
75%,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


In [13]:
sc = StandardScaler()
df_scaled = sc.fit_transform(df_cleaned)
df_scaled.shape

(1586599, 4)

In [14]:
joblib.dump(sc, 'models/standard_scaler.joblib')

['models/standard_scaler.joblib']

In [15]:
# split data
X_data, X_test, y_data, y_test = train_test_split(df_scaled, enc_target, stratify=enc_target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, stratify=y_data, test_size=0.15, random_state=42)

In [16]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

## Build Neural Network

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [18]:
from src.models.pytorch import PytorchMultiClass

model = PytorchMultiClass(X_train.shape[1])

In [19]:
from src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=4, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

In [20]:
len(np.unique(y_train))

104

In [21]:
device

device(type='cpu')

In [22]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=4, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)


In [23]:
criterion = nn.CrossEntropyLoss()

In [24]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [25]:
N_EPOCHS = 3
BATCH_SIZE = 16

In [26]:
from src.models.pytorch import train_classification, test_classification

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)

Epoch: 0
	(train)	|	Loss: 0.2858	|	Acc: 8.8%
	(valid)	|	Loss: 0.2857	|	Acc: 9.0%
Epoch: 1
	(train)	|	Loss: 0.2857	|	Acc: 8.9%
	(valid)	|	Loss: 0.2856	|	Acc: 9.0%
Epoch: 2
	(train)	|	Loss: 0.2857	|	Acc: 8.9%
	(valid)	|	Loss: 0.2856	|	Acc: 9.1%


In [27]:
y_pred_list = []

with torch.no_grad():
    model.eval()
    data = DataLoader(test_dataset, batch_size=1, collate_fn=None)
    for X_batch, _ in data:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        _, y_pred_tags = torch.max(y_test_pred, dim=-1)
        y_pred_list.append(y_pred_tags.cpu().numpy())

In [28]:
y_pred_list[:20]

[array([9]),
 array([1]),
 array([14]),
 array([12]),
 array([12]),
 array([12]),
 array([12]),
 array([12]),
 array([12]),
 array([9]),
 array([9]),
 array([12]),
 array([12]),
 array([14]),
 array([12]),
 array([12]),
 array([12]),
 array([12]),
 array([60]),
 array([9])]

In [40]:
torch.save(model.state_dict(), "models/pt_base_state_dict")

In [69]:
arch = model._modules
arch = {i: arch[i] for i in model._modules}
arch

{'layer_1': Linear(in_features=4, out_features=128, bias=True),
 'layer_out': Linear(in_features=128, out_features=104, bias=True),
 'softmax': Softmax(dim=1)}

In [47]:
[[print(k) for k,v in model._modules]]

TypeError: 'PytorchMultiClass' object is not iterable

In [72]:
from torchsummary import summary

In [81]:
model._modules

OrderedDict([('layer_1', Linear(in_features=4, out_features=128, bias=True)),
             ('layer_out',
              Linear(in_features=128, out_features=104, bias=True)),
             ('softmax', Softmax(dim=1))])

In [2]:
[print(i) for i in model._modules]

NameError: name 'model' is not defined